In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne 
from nice.markers import (KolmogorovComplexity, TimeLockedContrast, PowerSpectralDensityEstimator, 
                          PowerSpectralDensity, SymbolicMutualInformation, PermutationEntropy)

import pycsd

Using MNE with API > 0.19


In [3]:
def all_markers(epochs, tmin, tmax):
        """
        Computes all ther markers for given epochs.
        epochs: the epochs from which to compute the markers
        
        """
        
        

        # =============================================================================
        # SPECTRAL MARKERS
        # =============================================================================
          #PowerSpectralDensityL
        psds_params = dict(n_fft=4096, n_overlap=100, n_jobs='auto', nperseg=128)
        base_psd = PowerSpectralDensityEstimator(
            psd_method='welch', tmin=tmin, tmax=tmax, fmin=1., fmax=45.,
            psd_params=psds_params, comment='default')


            #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alphan')
        alpha.fit(epochs)
        from scipy.stats import trim_mean
        def trim_mean80(a, axis=0):
            return trim_mean(a, proportiontocut=.1, axis=axis)


        reduction_func = [{'axis': 'frequency', 'function': np.sum},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alpha')
        alpha.fit(epochs)
        dataalpha_n = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #alpha
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=False, comment='alpha')
        alpha.fit(epochs)
        dataalpha = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #delta normalized
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,normalize=True, comment='delta')
        delta.fit(epochs)
        datadelta_n = delta._reduce_to(reduction_func, target='epochs', picks=None)


        #delta
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4,normalize=False, comment='delta')
        delta.fit(epochs)
        datadelta = delta._reduce_to(reduction_func, target='epochs', picks=None)

        #theta normalized
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,normalize=True, comment='theta')
        theta.fit(epochs)
        datatheta_n = theta._reduce_to(reduction_func, target='epochs', picks=None)


        #theta
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8,normalize=False, comment='theta')
        theta.fit(epochs)
        datatheta = theta._reduce_to(reduction_func, target='epochs', picks=None)

        #gamma normalized
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,normalize=True, comment='gamma')
        gamma.fit(epochs)
        datagamma_n = gamma._reduce_to(reduction_func, target='epochs', picks=None)


        #gamma
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45,normalize=False, comment='theta')
        gamma.fit(epochs)
        datagamma = gamma._reduce_to(reduction_func, target='epochs', picks=None)

        #beta normalized
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30.,normalize=True, comment='beta')
        beta.fit(epochs)
        databetaa_n = beta._reduce_to(reduction_func, target='epochs', picks=None)


        #beta
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30,normalize=False, comment='beta')
        beta.fit(epochs)
        databeta = beta._reduce_to(reduction_func, target='epochs', picks=None)


        # =============================================================================
        # INFORMATION THEORY MARKERS
        # =============================================================================

        komplexity = KolmogorovComplexity(tmin=tmin, tmax=tmax, backend='openmp')
        komplexity.fit(epochs)
        komplexityobject=komplexity.data_ ###Object to save, number of channels*number of epochs, it's ndarray
        reduction_func= [{'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]


        datakomplexity = komplexity._reduce_to(reduction_func, target='epochs', picks=None)


        p_e = PermutationEntropy(tmin=tmin, tmax=tmax)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e = p_e._reduce_to(reduction_func, target='epochs', picks=None)

        # =============================================================================
        # wSMI MARKERS
        # =============================================================================
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=8, backend="openmp",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_

        reduction_func= [{'axis': 'channels_y', 'function': np.median},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        
        datawSMI = wSMI._reduce_to(reduction_func, target='epochs', picks=None)
        
        
        return {'wSMI':datawSMI, 'p_e':datap_e, 'k':datakomplexity, 'b':databeta,'b_n':databetaa_n, 'g':datagamma, 'g_n':datagamma_n, 't':datatheta,'t_n': datatheta_n , 'd':datadelta,
        'd_n':datadelta_n, 'a_n':dataalpha_n, 'a':dataalpha}


In [9]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'


all_participants = ['VP07','VP08','VP09', 'VP10','VP11','VP12','VP13','VP14','VP18','VP19','VP20','VP22','VP23','VP24','VP25','VP26','VP27','VP28','VP29','VP30','VP31','VP32','VP33','VP35','VP36','VP37']


path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
# path = '/Users/nicobruno/ownCloud/MW_eeg_data/minmarker/' #nico-mac

In [20]:
for i in all_participants:
    participant = i

    folder = path + participant +'/'
    epochs = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_clean_epo.fif')
    
    epochs.info['description'] = 'biosemi/64' #necessary for wSMI
    
    epochs =  epochs.pick_types(eeg = True) #EOGs break everything
    
    df_markers = pd.DataFrame()
    
    try:
        df_ = pd.DataFrame.from_dict(all_markers(epochs['SC/dMW/go/correct'], 0, 0.6))
        df_['probe'] = 'SC'
        df_['mind'] = 'dMW'
        df_markers = df_markers.append(df_)
    except:
        pass
    try:
        df_ = pd.DataFrame.from_dict(all_markers(epochs['SC/sMW/go/correct'], 0, 0.6))
        df_['probe'] = 'SC'
        df_['mind'] = 'sMW'
        df_markers = df_markers.append(df_)
    except:
        pass

    try:
        df_ = pd.DataFrame.from_dict(all_markers(epochs['PC/dMW/go/correct'], 0, 0.6))
        df_['probe'] = 'PC'
        df_['mind'] = 'dMW'
        df_markers = df_markers.append(df_)
    except:
        pass
    
    try:
        df_ = pd.DataFrame.from_dict(all_markers(epochs['PC/sMW/go/correct'], 0, 0.6))
        df_['probe'] = 'PC'
        df_['mind'] = 'sMW'
        df_markers = df_markers.append(df_)
    except:
        pass
    try:
        df_ = pd.DataFrame.from_dict(all_markers(epochs['PC/on-task/go/correct'], 0, 0.6))
        df_['probe'] = 'PC'
        df_['mind'] = 'on-task'
        df_markers = df_markers.append(df_)
    except:
        pass
    
    df_markers.to_csv(folder+ participant + '_' + epoch_type + '_all_marker.csv')


Reading /media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/VP07/VP07_evoked_ar_clean_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
123 matching events found
No baseline correction applied
0 projection items activated
Autodetected number of jobs 24
Effective window size : 16.384 (s)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    3.5s remaining:   10.4s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    3.6s remaining:    4.2s
[Parallel(n_jobs=24)]: Done  16 out of  24 | elapsed:    3.6s remaining:    1.8s


Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']


[Parallel(n_jobs=24)]: Done  21 out of  24 | elapsed:    3.7s remaining:    0.5s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    3.9s finished
/home/nicolas.bruno/anaconda3/lib/python3.7/site-packages/nice-0.1.dev0-py3.7.egg/nice/markers/spectral.py:128: RuntimeWarning: invalid value encountered in true_divide
  self.data_norm_ = self.data_ / self.data_.sum(axis=-1)[..., None]
/home/nicolas.bruno/anaconda3/lib/python3.7/site-packages/nice-0.1.dev0-py3.7.egg/nice/markers/spectral.py:220: RuntimeWarning: divide by zero encountered in log10
  out = 10 * np.log10(out)


Running KolmogorovComplexity
Elapsed time 0.007952451705932617 sec
Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 10.42 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 2
Computing CSD
Using 10-5 locations for CSD
Using 2 jobs


...

[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    3.8s finished


Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 2
Computing CSD
Using 10-5 locations for CSD
Using 2 jobs


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering  at 10.42 Hz
Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']


[Parallel(n_jobs=2)]: Done  15 out of  15 | elapsed:    1.7s finished


In [17]:
try:
    df_ = pd.DataFrame.from_dict(all_markers(epochs['SC/sMW/go/correct'], 0, 0.6))
    df_['probe'] = 'SC'
    df_['mind'] = 'sMW'
    df_markers.append(df_)
except:
    pass

Autodetected number of jobs 24
Effective window size : 16.384 (s)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    3.4s remaining:   10.2s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    3.5s remaining:    4.1s
[Parallel(n_jobs=24)]: Done  16 out of  24 | elapsed:    3.5s remaining:    1.8s


Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Running KolmogorovComplexity


[Parallel(n_jobs=24)]: Done  21 out of  24 | elapsed:    3.6s remaining:    0.5s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    3.8s finished


Elapsed time 0.05859518051147461 sec
Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 10.42 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 2
Computing CSD
Using 10-5 locations for CSD
Using 2 jobs


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering  at 10.42 Hz


[Parallel(n_jobs=2)]: Done  18 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done  38 out of  38 | elapsed:    1.7s finished


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
